# Ideal Diatomic Gas Model
Code by Scott Colton <br/>
Writeup by Isabelle Parker <br/>

The main question that this project seeks to answer is how diatomic ideal gas molecules interact differently (rotational motion, particle-wall collisions, particle-particle collisions) than the monatomic particles that were discussed and simulated in prior classes. <br/>

#### We seek to find an answer through this project by doing the following: 
- Creating structally diatomic molecules.
- Giving each diatom a random rotational velocity vector $\vec{\omega}$ and a random translational motion vector $\vec{s}$. 
- Modelling each diatom as a rotor in order to make physically accurate particle-wall collisions.
- (Potentially) Model molecule-molecule collisions as rotor-rotor collisions.

# Constructing Diatoms

For the creation of diatoms, there were several options that we could've taken. <br/>

The first option that we went for is the one that most people would probably think of: two spheres connected to each other. <br/>
This option ended up giving us a hard time figuring out the math needed to calculate the vectors needed to calculate forces needed to perform collisions, so we went for a modified option. <br/>

The modification we made to the two-sphere model was an added cylinder; positioned at the center of mass for the diatoms, and extending to the centerpoint of one of the atoms. The process of building the diatoms is shown in the first for loop of in the main cell.

# Giving Rotational and Translation Motions

Each diatom is given a random rotational and translational motion vector shortly after creation. <br/>
The speeds given for each direction is a random value between -1 and 1 multiplied by the root mean speed of an ideal gas, which is the following: <br/>

$$ s = \sqrt{\frac{3k_{b}T}{m}} $$



## Diatom - Wall Collisions

In order to accurately model the diatom-wall collision, we must treat them as rotors, which is quite commonplace in physical chemistry models of diatoms. <br/>

This is where the spheres that were attached to the mathematical "diatom" that was mentioned in the *Constructing Diatoms* cell. <br/>
When one of the spheres touches the wall (position comes within one radius of the wall), the *calc_wall* function is called. <br/> 
This function will find the force that is imparted on the diatom by the wall. One of the vectors needed ($\vec{r}_{col}) was easy to visualize, but hard to figure out the math for. <br/>
This vector was made trivial to calculate with the following equivalency:

$$ \vec{r}_{col} = \vec{r}_{cm->col.atom} + \vec{r}_{col.atom->wall} $$

Now that we knew the vector from the center of mass to the point of impact, the torque and force exerted on the diatom were much easier to find.

The torque exerted on a diatom as it collides against a wall is the following: <br/>

$$ \vec{\tau_{cm}} = \vec{r_{col}} \times \vec{F} $$
<br/>
with $\vec{r_{col}}$ being the vector pointing from the diatom's center of mass to the point of collision with the wall.
<br/>

The force imparted on a atom in this case is calculated in the *calc_wall* function, where: <br/>

$$ \vec{F} = -\frac{2\vec{v}_{particle}}{dt}  m_{particle} $$ <br/>

Being the force required to send the colliding atom back in opposite direction, giving the diatom an overal negative $\omega$ from before impact. <br/>



The resultant change in the diatom's anuglar momentum is: <br/>

$$ \Delta\vec{L} = \vec{\tau_{cm}}\Delta t $$ <br/>
$$ \vec{L_{f}} = \vec{L_{i}} + \Delta\vec{L} $$ <br/>

With this, we now have working diatom-wall collisions!

# Ability of the program to simulate reality

While this program is able to simulate diatom-wall collisions, it is not able to do so perfectly, and are assuming completely elastic collisions. <br/> 
This is due to the lack of friction of the wall. <br/>

As much as we wanted to also simulate diatom-diatom collisions, we were unable to do so within the time span of this project. However, the program already includes a scalable number of particles, which would allow for the simulation of multi-diatom collisions once the physics and math of these types of collisions are implemented in the future.

In [1]:
from vpython import *
import numpy as np
import random as rand
import matplotlib.pyplot as plt

<IPython.core.display.Javascript object>

In [4]:
def calc_wall(j, i, dire):
    if dire == 1:
        rcol = axes[j].pos - (particles[j][i].pos + R * vec(1,0,0))
        Fhat = vec(-1,0,0)
        
    if dire == -1:
        rcol = axes[j].pos - (particles[j][i].pos - R * vec(1,0,0))
        Fhat = vec(1,0,0)
        
    if dire == 2:
        rcol = axes[j].pos - (particles[j][i].pos + R * vec(0,1,0))
        Fhat = vec(0,-1,0)
        
    if dire == -2:
        rcol = axes[j].pos - (particles[j][i].pos -R * vec(0,1,0))
        Fhat = vec(0,1,0)
        
    if dire == 3:
        rcol = axes[j].pos - (particles[j][i].pos + R * vec(0,0,1))
        Fhat = vec(0,0,-1)
        
    if dire == -3:
        rcol = axes[j].pos - (particles[j][i].pos - R * vec(0,0,1))
        Fhat = vec(0,0,1)
           
    Fmag = -(mag(rcol) * mag(omega[j])) * m / dt  #This is the force that would be required to drop the velocity to 0
                                                  #and send the particle back at the same speed in the other direction
    F = Fmag * Fhat
    tau = cross(rcol,F)
    dL = tau * dt
    Li = I * omega[j]
    Lf = Li + dL
    omegaf = Lf/I
    
    return(omegaf)



In [5]:
scene = canvas(title = "Diatoms in a Box")

R = .5e-10  #radius of an atom
m = 1.7e-27 #mass of an atom
L = 40 * R
thick = L/100 #Thickness and positions of wall
Q = 3 * R
k = 1.4e-23
T = 300
s =  np.sqrt(2*3/2*k*T/m) #I like this as an initial speed

N = 2 #number of particles

particles = []
molecules = []
vcm = []
omega = []
axes = []
I = 2 * m * R**2
omegaf = 0


#The next loop in the code creates 2 particles that are touching and coupled in an array. The molecules have a random
#position and axis. The loop also assigns the diatoms a random center of mass velocity and angular velocity

for i in range(0,N):
    if i == 0:
        atom1 = sphere(pos = L/2*vec(rand.uniform(-0.9,0.9),rand.uniform(-0.9,0.9),rand.uniform(-0.9,0.9)), radius = R, color = color.cyan, make_trail = True, retain = 50)
        r2 = rand.random()*R
        r3 = rand.random()*R
        r4 = abs((2*R)**2 - r2**2 - r3**2)
        a = np.sqrt(r4)
        atom2 = sphere(pos = atom1.pos - vec(r2,a,r3), radius = R, color = color.cyan, make_trail = True, retain = 50)
        particles.append([atom1,atom2])
        rod = cylinder(pos = vec((particles[i][1].pos.x + particles[i][0].pos.x)/2,(particles[i][1].pos.y + particles[i][0].pos.y)/2,(particles[i][1].pos.z + particles[i][0].pos.z)/2), axis = particles[i][0].pos - particles[i][1].pos, radius = R/4, visible = False)
        axes.append(rod)
        vcm.append(s*hat(vec(rand.uniform(-1,1), rand.uniform(-1,1), rand.uniform(-1,1))))
        omega.append(360*hat(vec(rand.uniform(-1,1), rand.uniform(-1,1), rand.uniform(-1,1))))
    else:
        atom1 = sphere(pos = L/2*vec(rand.uniform(-0.9,0.9),rand.uniform(-0.9,0.9),rand.uniform(-0.9,0.9)), radius = R, color = color.red)
        r2 = rand.random()*R
        r3 = rand.random()*R
        r4 = abs((2*R)**2 - r2**2 - r3**2)
        a = np.sqrt(r4)
        atom2 = sphere(pos = atom1.pos - vec(r2,a,r3), radius = R, color = color.red)
        particles.append([atom1,atom2])
        rod = cylinder(pos = vec((particles[i][1].pos.x + particles[i][0].pos.x)/2,(particles[i][1].pos.y + particles[i][0].pos.y)/2,(particles[i][1].pos.z + particles[i][0].pos.z)/2), axis = particles[i][0].pos - particles[i][1].pos, radius = R/4, visible = False)
        axes.append(rod)
        vcm.append(s*hat(vec(rand.uniform(-1,1), rand.uniform(-1,1), rand.uniform(-1,1))))
        omega.append(360*hat(vec(rand.uniform(-1,1), rand.uniform(-1,1), rand.uniform(-1,1))))
    
#Creating the walls of the box

Lwall = box(pos = vec(-L/2, 0, 0), size = vec(thick, L, L), color=color.white)
Rwall = box(pos = vec(L/2, 0, 0), size = vec(thick, L, L), color=color.white)
Bwall = box(pos = vec(0, -L/2, 0), size = vec(L, thick, L), color=color.white)
Twall = box(pos = vec(0, L/2, 0), size = vec(L, thick, L), color=color.white)
Zwall = box(pos = vec(0, 0, -L/2), size = vec(L, L, thick), color=color.white)

#Adding in the motion and the time steps

dt = R/s/10
Nstep = 3e3
t = 0

scene.pause()

while t < dt * Nstep:
    rate(100)
    for j in range(N):
        
        axes[j].pos = axes[j].pos + vcm[j] * dt
        
        #Do the actual math to figure out what the change in axis and fix the check collisions with the walls with the new
        #rod
        
        dtheta = omega[j] * dt / R
        
        Rx = [[1, 0, 0], [0, np.cos(dtheta.x), -np.sin(dtheta.x)], [0, np.sin(dtheta.x), np.cos(dtheta.x)]]
        Ry = [[np.cos(dtheta.y), 0, np.sin(dtheta.y)], [0, 1, 0], [-np.sin(dtheta.y), 0, np.cos(dtheta.y)]]
        Rz = [[np.cos(dtheta.z), -np.sin(dtheta.z), 0], [np.sin(dtheta.z), np.cos(dtheta.z), 0], [0, 0, 1]]
        
        
        newaxis = np.array([axes[j].axis.x, axes[j].axis.y, axes[j].axis.z])
        newaxis = np.dot(Rx,newaxis)
        newaxis = np.dot(Ry,newaxis)
        newaxis = np.dot(Rz,newaxis)
        axes[j].axis.x = newaxis[0]
        axes[j].axis.y = newaxis[1]
        axes[j].axis.z = newaxis[2]
        
        particles[j][0].pos = axes[j].pos - axes[j].axis/2
        particles[j][1].pos = axes[j].pos + axes[j].axis/2
        
        
        
        
        #Checking the center of mass collisions with the walls this is being done separate from the individual particles
        
        #This is checking to see if the individual particles are hitting the walls separate to the center of mass
        
        for i in range(0,1):
            if particles[j][i].pos.x  > L/2 - R:
                d = 1
                omega[j] = calc_wall(j,i,d)
                
            if particles[j][i].pos.x  < -L/2 + R:
                d = -1
                omega[j] = calc_wall(j,i,d)
            
            if particles[j][i].pos.y > L/2 - R:
                d = 2
                omega[j] = calc_wall(j,i,d)
                
            if particles[j][i].pos.y < -L/2 + R:
                d = -2
                omega[j] = calc_wall(j,i,d)
                        
            if particles[j][i].pos.z > L/2 - R:
                d = 3
                omega[j] = calc_wall(j,i,d)
                
            if particles[j][i].pos.z < -L/2 + R:
                d = -3
                omega[j] = calc_wall(j,i,d)
                    
        #This is testing to see if the center of mass is colliding with the wall
        
        if axes[j].pos.x > L/2:
            vcm[j].x = -vcm[j].x
            
        if axes[j].pos.x < -L/2:
            vcm[j].x = abs(vcm[j].x)
            
        if axes[j].pos.y > L/2:
            vcm[j].y = -vcm[j].y
            
        if axes[j].pos.y < -L/2:
            vcm[j].y = abs(vcm[j].y)
            
        if axes[j].pos.z > L/2:
            vcm[j].z = -vcm[j].z
            
        if axes[j].pos.z < -L/2:
            vcm[j].z = abs(vcm[j].z)
        
    t = t + dt
    
    
#The simulation looks very weird and I think it is because there is no friction from the walls that would cause the
#motion that we would expect to see from diatoms in a box


#It is also glitching a bit from having the new omega be calculated multiple times in a row which could be fixed
#With some sort of timer that only lets particles hit certain walls once in n number of time steps


<IPython.core.display.Javascript object>